# 02. Backtest Cohort Split / Backtest Chia Cohort

Notebook này thực hiện backtest với phương pháp chia cohort train/test.

This notebook performs backtesting using train/test cohort split methodology.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# Import modules / Nhập các module
from config import CFG, SEGMENT_COLS, BUCKETS_CANON, BUCKETS_30P, ABSORBING_BASE, MAX_MOB, DENOM_LEVEL, K_CLIP
from config import PRIOR_STRENGTH_COARSE, PRIOR_STRENGTH_FULL, WEIGHT_MODE, MIN_COUNT, TAIL_POOL_START
from data_io import load_parquet
from transitions import prepare_transitions, estimate_transition_matrices
from forecast import build_initial_vectors, forecast
from calibration import fit_del_curve_factors
from metrics import compute_del_from_snapshot, compute_del_from_forecast, make_mixed_report
from backtest import split_cohorts, compute_backtest_metrics
from export import export_to_excel

## 1. Load Data / Tải Dữ liệu

In [ ]:
df = load_parquet('../Oct25.parquet')
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
display(df.head())

## 2. Train/Test Split / Chia Train/Test

In [ ]:
train_df, test_df = split_cohorts(df, CFG, train_ratio=0.7)

train_cohorts = sorted(train_df[CFG['cohort']].unique())
test_cohorts = sorted(test_df[CFG['cohort']].unique())

print(f"Train cohorts ({len(train_cohorts)}): {train_cohorts}")
print(f"Test cohorts ({len(test_cohorts)}): {test_cohorts}")
print(f"\nTrain size: {len(train_df):,} rows")
print(f"Test size: {len(test_df):,} rows")

## 3. Train: Estimate Transition Matrices / Train: Ước lượng Ma trận

In [ ]:
# Prepare transitions from training data
df_trans = prepare_transitions(train_df, CFG, SEGMENT_COLS, BUCKETS_CANON, ABSORBING_BASE)
print(f"Transitions: {len(df_trans):,} records")

# Estimate matrices
segment_levels = [
    ("GLOBAL", []),
    ("COARSE", [SEGMENT_COLS[0]] if SEGMENT_COLS else []),
    ("FULL", SEGMENT_COLS),
]
prior_strengths = {"coarse": PRIOR_STRENGTH_COARSE, "full": PRIOR_STRENGTH_FULL}

transitions_dict, transitions_long_df, meta_df = estimate_transition_matrices(
    df_trans, CFG, BUCKETS_CANON, segment_levels,
    max_mob=MAX_MOB, weight_mode=WEIGHT_MODE, min_count=MIN_COUNT,
    prior_strengths=prior_strengths, tail_pool_start=TAIL_POOL_START
)

print(f"Total matrices: {len(transitions_dict)}")

## 4. Test: Baseline Forecast (No Calibration) / Test: Dự báo Cơ sở

In [ ]:
# Build initial vectors from test data
df_init, denom_map = build_initial_vectors(test_df, CFG, BUCKETS_CANON, SEGMENT_COLS, DENOM_LEVEL)
print(f"Initial vectors: {len(df_init)} records")

# Forecast
forecast_df = forecast(df_init, transitions_dict, BUCKETS_CANON, MAX_MOB)
print(f"Forecast: {len(forecast_df):,} records")

In [ ]:
# Compute actual DEL from test data
actual_del_long, _ = compute_del_from_snapshot(
    test_df, CFG, BUCKETS_30P, SEGMENT_COLS, MAX_MOB, DENOM_LEVEL
)

# Compute predicted DEL (baseline)
pred_del_long_baseline = compute_del_from_forecast(forecast_df, BUCKETS_30P, denom_map)

print(f"Actual DEL: {len(actual_del_long)} records")
print(f"Pred DEL (baseline): {len(pred_del_long_baseline)} records")

## 5. Baseline Metrics / Chỉ số Cơ sở

In [ ]:
metrics_baseline = compute_backtest_metrics(actual_del_long, pred_del_long_baseline, MAX_MOB)
print("Baseline Metrics by MOB:")
display(metrics_baseline)

## 6. Fit Calibration Factors / Fit Hệ số Hiệu chỉnh

In [ ]:
# Fit calibration factors on TRAINING data
train_actual, train_denom = compute_del_from_snapshot(
    train_df, CFG, BUCKETS_30P, SEGMENT_COLS, MAX_MOB, DENOM_LEVEL
)
train_init, _ = build_initial_vectors(train_df, CFG, BUCKETS_CANON, SEGMENT_COLS, DENOM_LEVEL)
train_forecast = forecast(train_init, transitions_dict, BUCKETS_CANON, MAX_MOB)
train_pred = compute_del_from_forecast(train_forecast, BUCKETS_30P, train_denom)

factors_df = fit_del_curve_factors(train_actual, train_pred, MAX_MOB, K_CLIP)
print("Calibration Factors:")
display(factors_df)

## 7. Plot: K-Factors by MOB / Biểu đồ: Hệ số K theo MOB

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(factors_df['mob'], factors_df['k'], marker='o', linewidth=2)
ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='k=1 (no adjustment)')
ax.axhline(y=K_CLIP[0], color='gray', linestyle=':', alpha=0.5, label=f'k_min={K_CLIP[0]}')
ax.axhline(y=K_CLIP[1], color='gray', linestyle=':', alpha=0.5, label=f'k_max={K_CLIP[1]}')
ax.set_xlabel('MOB')
ax.set_ylabel('K-Factor')
ax.set_title('Calibration K-Factors by MOB / Hệ số K theo MOB')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Calibrated Forecast (Optional) / Dự báo Đã Hiệu chỉnh

In [ ]:
# For simplicity, we use baseline forecast here
# Full calibration would apply k-factors to transition matrices or vectors
pred_del_long = pred_del_long_baseline.copy()

# Compute metrics
metrics_df = compute_backtest_metrics(actual_del_long, pred_del_long, MAX_MOB)
print("Final Metrics by MOB:")
display(metrics_df)

## 9. Plot: MAE by MOB / Biểu đồ: MAE theo MOB

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(metrics_df['mob'], metrics_df['mae'] * 100, alpha=0.7)
ax.set_xlabel('MOB')
ax.set_ylabel('MAE (%)')
ax.set_title('Mean Absolute Error by MOB / Sai số Tuyệt đối Trung bình theo MOB')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 10. Mixed Report / Báo cáo Hỗn hợp

In [ ]:
mixed_wide, flags_wide, actual_wide, forecast_wide = make_mixed_report(
    actual_del_long, pred_del_long, MAX_MOB
)

print("Mixed Report (first 10 rows):")
display(mixed_wide.head(10))

print("\nFlags (first 10 rows):")
display(flags_wide.head(10))

## 11. Export to Excel / Xuất ra Excel

In [ ]:
import os
os.makedirs('../out', exist_ok=True)

export_to_excel(
    '../out/backtest_report_oct25.xlsx',
    transitions_long_df,
    mixed_wide,
    flags_wide,
    actual_wide,
    forecast_wide,
    factors_df=factors_df,
    forecast_df=forecast_df,
    meta_df=meta_df
)
print("Export complete: ../out/backtest_report_oct25.xlsx")

## 12. Interpretation Guide / Hướng dẫn Diễn giải

### Cách đọc MAE/MAPE theo MOB / How to Read MAE/MAPE by MOB

**VI:**
- MAE (Mean Absolute Error): Sai số tuyệt đối trung bình giữa DEL thực tế và dự báo
- MAPE (Mean Absolute Percentage Error): Sai số phần trăm tuyệt đối trung bình
- MAE thấp ở MOB đầu (0-6) là bình thường vì DEL còn thấp
- MAE tăng ở MOB cao (12-24) cho thấy sai số tích lũy

**EN:**
- MAE (Mean Absolute Error): Average absolute difference between actual and forecast DEL
- MAPE (Mean Absolute Percentage Error): Average percentage error
- Low MAE at early MOBs (0-6) is normal since DEL is still low
- Increasing MAE at high MOBs (12-24) indicates compounding error

### Các Lỗi Thường gặp / Typical Failure Modes

**VI:**
1. MAE tăng đột biến ở MOB cao → Cần tail pooling hoặc calibration
2. MAPE > 50% → Mô hình không phù hợp với segment này
3. K-factor luôn ở biên (0.5 hoặc 2.0) → Dữ liệu train không đại diện

**EN:**
1. MAE spikes at high MOBs → Need tail pooling or calibration
2. MAPE > 50% → Model not suitable for this segment
3. K-factor always at bounds (0.5 or 2.0) → Training data not representative

### Bước Tiếp theo / Next Steps for Tuning

**VI:**
1. Điều chỉnh TAIL_POOL_START nếu MAE cao ở MOB cuối
2. Tăng PRIOR_STRENGTH nếu segment nhỏ có variance cao
3. Thử CALIBRATION_MODE="vector" nếu "matrix" không hiệu quả

**EN:**
1. Adjust TAIL_POOL_START if MAE is high at late MOBs
2. Increase PRIOR_STRENGTH if small segments have high variance
3. Try CALIBRATION_MODE="vector" if "matrix" is not effective